In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("json", data_files="..\datasets\copa-translated\\hi\\train.jsonl", \
                             split="train")

In [ ]:
val_dataset = load_dataset("json", data_files="..\datasets\copa-translated\\hi\\val.jsonl", \
                            split="train")

In [ ]:
test_dataset = load_dataset("json", data_files="..\datasets\copa-translated\\hi\\test.jsonl", \
                             split="train")

In [ ]:
from datasets import Dataset, DatasetDict

datasets = DatasetDict()
datasets['train'] = train_dataset
datasets['validation'] = val_dataset
datasets['test'] = test_dataset

In [ ]:
# from datasets import load_dataset

# datasets = load_dataset("indic_glue","copa.hi")

In [ ]:
datasets

In [ ]:
datasets["train"][:5]

In [ ]:
datasets["test"][:2]

In [ ]:
datasets.set_format("pandas")

In [ ]:
# get label counts for both classes
label_counts = datasets["train"]["label"].value_counts()
num_labels = (len(label_counts.keys()))

In [ ]:
label_counts

In [ ]:
datasets.reset_format()

In [ ]:
from transformers import PreTrainedTokenizerFast, AutoModelForSequenceClassification, AutoTokenizer

tokenizer = PreTrainedTokenizerFast.from_pretrained("../Hindi Pretraining/models/unigram/bert-base-pretrained-hindi")

In [ ]:
choice_names = ['choice1', 'choice2']

In [ ]:
def preprocess_function(examples):
    premise = [[context] * 2 for context in examples["premise"]]
    cause = [[f"{examples[choice][i]}" for choice in choice_names] for i,_ in enumerate(premise)]

    premise = sum(premise, [])
    cause = sum(cause, [])
    
#     print(premise)
#     print(cause)
    

    tokenized_examples = tokenizer(premise, cause, truncation=True)
#     print(len(tokenized_examples))
    return {k: [v[i : i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}
    #return tokenized_examples

In [ ]:
temp = preprocess_function(datasets["train"][:1])
temp

In [ ]:
datasets["train"][:1]

In [ ]:
for chunk in temp['input_ids'][0]:
    print(tokenizer.decode(chunk))

In [ ]:
tokenized_datasets = datasets.map(preprocess_function, batched=True)

In [ ]:
tokenized_datasets["train"][0]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer)

In [ ]:
tokenized_datasets["train"]

In [ ]:
temp_data = tokenized_datasets["train"].remove_columns(['premise', 'choice1', 'choice2', 'question', 'idx'])
temp_data

In [ ]:
samples = [temp_data[i] for i in range(1)]
temp = data_collator(samples)

In [ ]:
temp['input_ids'].size()

In [ ]:
temp_data = tokenized_datasets.remove_columns(['premise','choice1','choice2','question','idx'])

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import set_seed
set_seed(80)

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoModel

model = AutoModelForMultipleChoice.from_pretrained("../Hindi Pretraining/models/unigram/bert-base-pretrained-hindi")

In [ ]:
model

In [ ]:
# temp_data = tokenized_datasets.remove_columns(['premise','choice1','choice2','question','idx'])
# temp = [temp_data["train"][i]for i in range(5)]

In [ ]:
# batch = data_collator(temp)
# batch

In [ ]:
# for sample in batch["input_ids"].tolist():
#     for choice in sample:
#         print(tokenizer.decode(choice))

In [ ]:
# datasets["train"][:5]

In [ ]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_swag_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    #learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    #num_train_epochs=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    #weight_decay=0.04,
    fp16=True,
    metric_for_best_model = 'accuracy',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    #eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [ ]:
y_preds, y_true, _ = trainer.predict(tokenized_datasets["test"])

In [ ]:
y_preds = np.argmax(y_preds, axis=-1)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['choice1', 'choice2']

In [ ]:
import matplotlib.pyplot as plt
from seaborn import heatmap
from sklearn.metrics import confusion_matrix

#plot heatmap of confusion matrix
mat = confusion_matrix(y_true, y_preds)
heatmap(mat, cmap="Pastel1_r", fmt="d", xticklabels=target_names, yticklabels=target_names, annot=True)

#add overall title to plot
plt.title('Confusion matrix for COPA', fontsize = 12) # title with fontsize 20

In [ ]:
misclassified = [i for i in range(len(y_preds)) if ((y_preds[i] != y_true[i]) and (y_true[i]==0) and (y_preds[i]==1))]

In [ ]:
misclassified_dataset = test_dataset.select(misclassified)

In [ ]:
misclassified_dataset[2]

In [ ]:
model = AutoModelForMultipleChoice.from_pretrained("my_awesome_swag_model")